# Checking COCO dataset(.json) file


> annotation info가 들어있는 COCO 데이터셋을 시각적으로 확인한다.



In [ ]:
import pandas as pd
import json

json_file_path = "./dataset/train_answer.json"
with open(json_file_path,'r') as j:
    contents=json.loads(j.read())

In [ ]:
df_img = pd.json_normalize(contents['images'])
df_img.loc[:4]

In [ ]:
df_anno = pd.json_normalize(contents['annotations'])
df_anno.describe()

In [ ]:
df_anno.loc[:8]

# Modify COCO dataset


> Training을 위해 COCO dataset을 수정한다.



In [ ]:
# annotations <-- id 추가 & segmentation(list(list)로 변경)
for i in range(16848):
  seg_list = contents['annotations'][i]['segmentation']
  contents['annotations'][i].update({'segmentation':[seg_list]})
  contents['annotations'][i].update({'id':i+1})

In [ ]:
df_anno = pd.json_normalize(contents['annotations'])    
df_anno.loc[:8]

In [ ]:
# category list 추가
category_update = [{"supercategory": "estrus", "id" : 1, "name" : "no"}, {"supercategory": "estrus", "id" : 2, "name" : "yes"}]
contents.update({"categories" : category_update})

In [ ]:
contents["categories"]

In [ ]:
# 수정된 json 파일 저장
file_save_path = "./dataset/train_answer_modified.json"
with open(file_save_path,'w') as j:
    json.dump(contents, j)

# Generate dataset & Train a Model

> 수정한 데이터를 불러와 모델을 학습시킨다. 
  [참고코드1](https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-1-of-2-732712631047)



In [ ]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import random
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### For visualizing the outputs ###
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [ ]:
annFile = "./dataset/train_answer_modified.json"
coco=COCO(annFile)

In [ ]:
catIDs = coco.getCatIds()
cats = coco.loadCats(catIDs)

print(cats)

In [ ]:
# Define the classes (out of the 81) which you want to see. Others will not be shown.
filterClasses = ['no', 'yes']

# Fetch class IDs only corresponding to the filterClasses
catIds = coco.getCatIds(catNms=filterClasses) 
# Get all images containing the above Category IDs
imgIds = coco.getImgIds(catIds=catIds)
print("Number of images containing all the  classes:", len(imgIds))

# load and display a random image
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
I = io.imread('./dataset/train/{}'.format(img['file_name']))/255.0

plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
# Load and display instance annotations
plt.imshow(I)
plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
########## ALl POSSIBLE COMBINATIONS ########
classes = ['no', 'yes']

images = []
if classes!=None:
    # iterate for each individual class in the list
    for className in classes:
        # get all images containing given class
        catIds = coco.getCatIds(catNms=className)
        imgIds = coco.getImgIds(catIds=catIds)
        images += coco.loadImgs(imgIds)
else:
    imgIds = coco.getImgIds()
    images = coco.loadImgs(imgIds)
    
# Now, filter out the repeated images    
unique_images = []
for i in range(len(images)):
    if images[i] not in unique_images:
        unique_images.append(images[i])

dataset_size = len(unique_images)

print("Number of images containing the filter classes:", dataset_size)

In [ ]:
def getClassName(classID, cats):
    for i in range(len(cats)):
        if cats[i]['id']==classID:
            return cats[i]['name']
    return "None"

In [ ]:
#### GENERATE A SEGMENTATION MASK ####
filterClasses = ['no', 'yes']
mask = np.zeros((img['height'],img['width']))
for i in range(len(anns)):
    className = getClassName(anns[i]['category_id'], cats)
    pixel_value = filterClasses.index(className)+1
    mask = np.maximum(coco.annToMask(anns[i])*pixel_value, mask)
plt.imshow(mask)

In [ ]:
#### GENERATE A BINARY MASK ####
mask = np.zeros((img['height'],img['width']))
for i in range(len(anns)):
    mask = np.maximum(coco.annToMask(anns[i]), mask)
plt.imshow(mask)

[참고 링크2](https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-2-of-2-c0d1f593096a)

In [ ]:
def filterDataset(folder, classes=None):    
    # initialize COCO api for instance annotations
    annFile = folder
    coco = COCO(annFile)
    
    images = []
    if classes!=None:
        # iterate for each individual class in the list
        for className in classes:
            # get all images containing given categories
            catIds = coco.getCatIds(catNms=className)
            imgIds = coco.getImgIds(catIds=catIds)
            images += coco.loadImgs(imgIds)
    
    else:
        imgIds = coco.getImgIds()
        images = coco.loadImgs(imgIds)
    
    # Now, filter out the repeated images
    unique_images = []
    for i in range(len(images)):
        if images[i] not in unique_images:
            unique_images.append(images[i])
            
    random.shuffle(unique_images)
    dataset_size = len(unique_images)
    
    return unique_images, dataset_size, coco

In [ ]:
folder = './dataset/train_answer_modified.json'
classes = ['yes', 'no']

images, dataset_size, coco = filterDataset(folder, classes)

In [ ]:
def getImage(imageObj, img_folder, input_image_size):
    # Read and normalize an image
    train_img = io.imread(img_folder + '/' + imageObj['file_name'])/255.0
    # Resize
    train_img = cv2.resize(train_img, input_image_size)
    if (len(train_img.shape)==3 and train_img.shape[2]==3): # If it is a RGB 3 channel image
        return train_img
    else: # To handle a black and white image, increase dimensions to 3
        stacked_img = np.stack((train_img,)*3, axis=-1)
        return stacked_img
    
def getNormalMask(imageObj, classes, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    cats = coco.loadCats(catIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        className = getClassName(anns[a]['category_id'], cats)
        pixel_value = classes.index(className)+1
        new_mask = cv2.resize(coco.annToMask(anns[a])*pixel_value, input_image_size)
        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask  
    
def getBinaryMask(imageObj, coco, catIds, input_image_size):
    annIds = coco.getAnnIds(imageObj['id'], catIds=catIds, iscrowd=None)
    anns = coco.loadAnns(annIds)
    train_mask = np.zeros(input_image_size)
    for a in range(len(anns)):
        new_mask = cv2.resize(coco.annToMask(anns[a]), input_image_size)
        
        #Threshold because resizing may cause extraneous values
        new_mask[new_mask >= 0.5] = 1
        new_mask[new_mask < 0.5] = 0

        train_mask = np.maximum(new_mask, train_mask)

    # Add extra dimension for parity with train_img size [X * X * 3]
    train_mask = train_mask.reshape(input_image_size[0], input_image_size[1], 1)
    return train_mask


def dataGeneratorCoco(images, classes, coco, folder, 
                      input_image_size=(224,224), batch_size=4, mode="train", val_idx=300, mask_type='binary'):
    img_folder = folder
    dataset_size = len(images)
    catIds = coco.getCatIds(catNms=classes)
    
    if mode == "train":
      c=0
    else:
      c=val_idx

    while(True):
        img = np.zeros((batch_size, input_image_size[0], input_image_size[1], 3)).astype('float')
        mask = np.zeros((batch_size, input_image_size[0], input_image_size[1], 1)).astype('float')

        for i in range(c, c+batch_size): #initially from 0 to batch_size, when c = 0
            imageObj = images[i]
            
            ### Retrieve Image ###
            train_img = getImage(imageObj, img_folder, input_image_size)
            
            ### Create Mask ###
            if mask_type=="binary":
                train_mask = getBinaryMask(imageObj, coco, catIds, input_image_size)
            
            elif mask_type=="normal":
                train_mask = getNormalMask(imageObj, classes, coco, catIds, input_image_size)                
            
            # Add to respective batch sized arrays
            img[i-c] = train_img
            mask[i-c] = train_mask
            
        c+=batch_size
        if(c + batch_size >= dataset_size):
            if mode == "train":
              c=0
            else:
              c=val_idx
            random.shuffle(images)
        yield img, mask

In [ ]:
batch_size = 4
input_image_size = (224,224)
mask_type = 'binary'
img_folder = "./dataset/train/"
val_gen = dataGeneratorCoco(images, classes, coco, img_folder, 
                            input_image_size, batch_size, mask_type)

In [ ]:
def visualizeGenerator(gen):
    # Iterate the generator to get image and mask batches
    img, mask = next(gen)
 
    fig = plt.figure(figsize=(20, 10))
    outerGrid = gridspec.GridSpec(1, 2, wspace=0.1, hspace=0.1)
   
    for i in range(2):        
        innerGrid = gridspec.GridSpecFromSubplotSpec(2, 2, subplot_spec=outerGrid[i], wspace=0.05, hspace=0.05)

        for j in range(4):
            ax = plt.Subplot(fig, innerGrid[j])
            if(i==1):
                ax.imshow(img[j]);
            else:
                ax.imshow(mask[j][:,:,0]);
                
            ax.axis('off')
            fig.add_subplot(ax)
    plt.show()

In [ ]:
visualizeGenerator(val_gen)

In [ ]:
def augmentationsGenerator(gen, augGeneratorArgs, seed=None):
    # Initialize the image data generator with args provided
    image_gen = ImageDataGenerator(**augGeneratorArgs)
    
    # Remove the brightness argument for the mask. Spatial arguments similar to image.
    augGeneratorArgs_mask = augGeneratorArgs.copy()
    _ = augGeneratorArgs_mask.pop('brightness_range', None)
    # Initialize the mask data generator with modified args
    mask_gen = ImageDataGenerator(**augGeneratorArgs_mask)
    
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    
    for img, mask in gen:
        seed = np.random.choice(range(9999))
        # keep the seeds syncronized otherwise the augmentation of the images 
        # will end up different from the augmentation of the masks
        g_x = image_gen.flow(255*img, 
                             batch_size = img.shape[0], 
                             seed = seed, 
                             shuffle=True)
        g_y = mask_gen.flow(mask, 
                             batch_size = mask.shape[0], 
                             seed = seed, 
                             shuffle=True)
        
        img_aug = next(g_x)/255.0
        mask_aug = next(g_y)
                   
        yield img_aug, mask_aug

In [ ]:
augGeneratorArgs = dict(featurewise_center = False, 
                        samplewise_center = False,
                        rotation_range = 5, 
                        width_shift_range = 0.01, 
                        height_shift_range = 0.01, 
                        brightness_range = (0.8,1.2),
                        shear_range = 0.01,
                        zoom_range = [1, 1.25],  
                        horizontal_flip = True, 
                        vertical_flip = False,
                        fill_mode = 'reflect',
                        data_format = 'channels_last')

# Call the function with the arguments
aug_gen = augmentationsGenerator(val_gen, augGeneratorArgs)

In [ ]:
visualizeGenerator(aug_gen)

# Train segmentation model

In [ ]:
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from tensorflow import keras
from keras.layers.core import Dropout, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import CSVLogger
from keras import backend as K

In [ ]:
#-> Create filtered train dataset (using filterDataset()) 
#-> Create filtered val dataset (using filterDataset()) 

#-> Create train generator (using dataGeneratorCoco()) 
#-> Create train generator (using dataGeneratorCoco()) 
batch_size = 10
train_batch_size = 200
val_batch_size = 200

input_image_size = (224,224)
mask_type = 'binary'
img_folder = "./dataset/train/"
train_gen = dataGeneratorCoco(images, classes, coco, img_folder, 
                            input_image_size=input_image_size, batch_size=train_batch_size, mode="train", val_idx=0, mask_type=mask_type)
val_gen = dataGeneratorCoco(images, classes, coco, img_folder, 
                            input_image_size=input_image_size, batch_size=val_batch_size, mode="val", val_idx=300, mask_type=mask_type)



# Set your parameters
n_epochs = 20
steps_per_epoch = train_batch_size // batch_size
validation_steps = val_batch_size // batch_size

model = keras.Sequential()
model.add(keras.Input(shape=(224, 224, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding = 'same'))
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding = 'same'))
print(model.summary())


# Compile your model first
model.compile(loss = "mse", optimizer = "adam", metrics=['accuracy'])

model_checkpoint = ModelCheckpoint(
        "segment_model_hjk.hdf5",
        monitor="val_loss",
        verbose=1,
        save_best_only=True,
        save_weights_only=True,
    )

stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=1, mode='min')
csv_logger = CSVLogger(
        "/segment_model_hjk.log",
        separator=",",
        append=True,
    )

print("Training is going to start in 3... 2... 1... ")

H = model.fit(
        x = train_gen,
        steps_per_epoch=steps_per_epoch,
        validation_data = val_gen,
        validation_steps=validation_steps,
        epochs=n_epochs,
        verbose = True,
        callbacks=[model_checkpoint, stopping, csv_logger],
    )